In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import nibabel as nib

##### FUNÇÃO DE OCLUSÃO

In [ ]:
def occlusion_sensitivity(model, image, block_size=(15, 15, 3), stride=8):    
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    
    # Predição original
    original_pred = model(image, training=False).numpy()[0, 1]  # Probabilidade para Alzheimer

    # Criar um heatmap vazio
    heatmap = np.zeros(image.shape[1:-1])  # Shape (145, 182, 7)

    # Iterar sobre a imagem com stride
    for x in range(0, image.shape[1] - block_size[0] + 1, stride):
        for y in range(0, image.shape[2] - block_size[1], stride):
            for z in range(0, image.shape[3] - block_size[2]+1, 1):
                occluded = tf.identity(image)

                # Aplicar oclusão via slicing (zera um bloco da imagem)
                occluded_numpy = occluded.numpy()  # Converte para numpy para manipulação mais fácil
                occluded_numpy[:, x:x+block_size[0], y:y+block_size[1], z:z+block_size[2], :] = 0
                occluded = tf.convert_to_tensor(occluded_numpy, dtype=tf.float32)

                # Fazer predição com a região ocluída
                occluded_pred = model(occluded, training=False).numpy()[0, 1]

                # Calcular importância da região
                importance = original_pred - occluded_pred
                heatmap[x:x+block_size[0], y:y+block_size[1], z:z+block_size[2]] += importance

    return heatmap

##### CARREGAR DADOS

In [ ]:
img_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/processed_7_slices_data/7_slices_axial/validation/ad/I56644.nii.gz"
image = nib.load(img_path).get_fdata()
image= tf.cast(image, dtype=tf.float32)
image = image[np.newaxis, ..., np.newaxis]

model_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/processed_7_slices_data/7_slices_axial/results/3d/axial_certo/test_5/binary_classifier_120_epochs_batch_64_5_classes.keras"
model = tf.keras.models.load_model(model_path)

##### GERAR MAPA DE OCLUSÃO

In [ ]:
bloco = 1
slices = 1
heatmap = occlusion_sensitivity(model, image, block_size=(bloco, bloco, slices), stride=(bloco))

##### PLOTAR MAPA DE OCLUSÃO

In [ ]:
fig, axes = plt.subplots(1, 7, figsize=(20, 5))  

for z in range(7):
    z_slice = z  

    # Normalizar o heatmap para ficar entre 0 e 1
    heatmap_normalized = (heatmap[:, :, z_slice] - np.min(heatmap[:, :, z_slice])) / \
                         (np.max(heatmap[:, :, z_slice]) - np.min(heatmap[:, :, z_slice]))

    # Plotar a imagem original em escala de cinza
    axes[z].imshow(image[0, :, :, z_slice, 0], cmap='gray', alpha=1)

    # Sobrepor o heatmap com transparência
    axes[z].imshow(heatmap_normalized, cmap='hot', alpha=0.5)  

    # Título da fatia
    axes[z].set_title(f"Slice {z}", fontsize=12)
    axes[z].axis("off")  

plt.suptitle("Oclusão Sensitivity", fontsize=16)
plt.subplots_adjust(top=1.3)  # Ajusta o espaço entre o título e as imagens
plt.show()